In [ ]:
%matplotlib inline
from ROOT import TFile, TTree
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from root_numpy import root2array

In [ ]:
#These files are the combined output from MCShowerStudy, MCTrackStudy, CosmicValidation, and CCSingleE run scripts.
#I make the first three at once with one run script in the mac/ directory, then I make the CCSingleE output
#and hadd it to the first.
filebase = '/Users/davidkaleko/larlite/UserDev/KalekoAna/scratch_ana/mac/data/cry_corsika_comp_'
filenames = { 'kazucry' : 'kazuCry.root', 
             'mattcorsika' : 'mattbass_corsika.root',
             'oldcry' : 'oscCry.root',
             'oldcorsikaNoOB' : 'davidccorsika_noOB.root'
            }

mcstreename = 'mcs_tree'
mcttreename = 'mct_tree'
erttreename = 'cos_tree'
ccstreename = 'cosmicShowers'

labels = { 'kazucry' : 'CRY: v04_29_02 Updated Physics List', 
             'mattcorsika' : 'Corsika: v04_30_00 (newest, no OB)',
             'oldcry' : 'CRY: Old Osc Group Sample',
             'oldcorsikaNoOB' : 'Corsika: v04_14_00 (oldish, no OB)'
          }
nevents = { 'kazucry' : 9625.,
            'mattcorsika' : 19000.,
            'oldcry' : 13100.,
            'oldcorsikaNoOB' : 20000.
          }
exposuretimes = { 'kazucry' : 6.8,
                'mattcorsika' : 7.4,
                'oldcry' : 6.4,
                'oldcorsikaNoOB' : 6.4 }

colors = {'kazucry' : '#FF3366', 
             'mattcorsika' : '#33FF66',
             'oldcry' : '#FFCC33',
             'oldcorsikaNoOB' : '#6633FF'
         }

def cosWeight(key):
    return 211000./(exposuretimes[key]*nevents[key])

In [ ]:
dfs = {}
for key, filename in filenames.iteritems():
   
    dfs[key] = { 'mcs' : pd.DataFrame( root2array( filebase + filename, mcstreename ) ),
                 'mct' : pd.DataFrame( root2array( filebase + filename, mcttreename ) ),
                 'ert' : pd.DataFrame( root2array( filebase + filename, erttreename ) ),
                 'ccs' : pd.DataFrame( root2array( filebase + filename, ccstreename ) ) 
               }

In [ ]:
def gen_histo(mykey = '', 
              mytype = '', 
              myvar = '', 
              myquery = '', 
              mybinning = np.linspace(0,1e6,200),
              myunitfactor = 1.,
              myweight = 1.):
    
    if not myvar: 
        print "wtf enter a variable"
        return False
    
    mydf = dfs[mykey][mytype].query('%s'%myquery) if myquery else dfs[mykey][mytype]
    myweights = np.ones(mydf.shape[0])*myweight
    return  np.histogram(mydf[myvar]/myunitfactor, bins=mybinning, weights = myweights)

In [ ]:
def plot_hist(key,myhisto, mytitle = '', myxlabel = '', myylabel = '', myyscale = '', myymax = 0, mysubplot = 0, myweight=0):

    hist, bins = myhisto
    if not myweight:
        plotlabel='%s: %0.2f Entries Per Evt'%(labels[key],float(sum(hist))/nevents[key])
    else:
        plotlabel='%s: %0.2f Entries'%(labels[key],sum(hist))
    
    plt.subplot(mysubplot)
    plt.bar(bins[:-1],hist,
            width=bins[1]-bins[0],
            label=plotlabel,
            alpha=0.7,
            color = colors[key])
    
    plt.xlim(bins[0],bins[-1])
    if myymax:
        plt.ylim([0,myymax])
    plt.title(mytitle)#,fontsize=20)
    plt.ylabel(myylabel)#,fontsize=20)
    plt.xlabel(myxlabel)#,fontsize=20)
    plt.yscale(myyscale)
    plt.legend()
    plt.grid(True)

In [ ]:
def lowLevelPlotsCompareSamples(keylist):
    subplots = range(221,221+len(keylist))
    ### MCShower Deposited Energy Plots
    fig = plt.figure(figsize=(20,12)) 
    for x in xrange(len(keylist)):
        mykey = keylist[x]
        mybins = np.linspace(0,3,200)
        myhisto = gen_histo(mykey = mykey, 
                            mytype = 'mcs', 
                            myvar = 'detprof_E', 
                            myquery = 'detprof_E > 0', 
                            mybinning = mybins,
                            myunitfactor = 1000.)
           
        plot_hist(mykey,
                  myhisto, 
                  mytitle = 'MCShower Deposited Energy (>0): %s'%mykey,
                  myxlabel = 'MCShower Deposited Energy [GeV]',
                  myylabel = '# of MCShowers',
                  myyscale = 'log',
                  mysubplot = subplots[x])

    ### MCShower Deposited Energy Plots (zoomed)
    fig = plt.figure(figsize=(20,12))
    for x in xrange(len(keylist)):
        mykey = keylist[x]
        mybins = np.linspace(0,0.1,200)
        myhisto = gen_histo(mykey = mykey, 
                            mytype = 'mcs', 
                            myvar = 'detprof_E', 
                            myquery = 'detprof_E > 0', 
                            mybinning = mybins,
                            myunitfactor = 1000.)

        plot_hist(mykey,
                  myhisto, 
                  mytitle = 'MCShower Deposited Energy (zoomed in) (>0): %s'%mykey,
                  myxlabel = 'MCShower Deposited Energy [GeV]',
                  myylabel = '# of MCShowers',
                  myyscale = 'log',
                  mysubplot = subplots[x])

    ### MCTrack Timing Plots
    fig = plt.figure(figsize=(20,12))
    for x in xrange(len(keylist)):
        mykey = keylist[x]
        mybins = np.linspace(-4,5,200)
        myhisto = gen_histo(mykey = mykey, 
                            mytype = 'mct', 
                            myvar = 'start_t', 
                            myquery = '', 
                            mybinning = mybins,
                            myunitfactor = 1e6)

        plot_hist(mykey,
                  myhisto, 
                  mytitle = 'MCTrack Start Time: %s'%mykey,
                  myxlabel = 'Start time (ms)',
                  myylabel = '# of MCTracks',
                  myyscale = 'linear',
                  mysubplot = subplots[x])


In [ ]:
#['_is_track', '_true_primary', '_true_secondary', '_true_orphan', 
#'_reco_primary', '_reco_secondary', '_reco_orphan', '_pdg_parent', 
#'_pdg_self', '_perp_dist_top_wall', '_orphan_dist', '_energy', 
#'_trk_length', '_start_x_vtx', '_start_y_vtx', '_start_z_vtx', 
#'_start_contained_in_TPC', '_trackend_contained_in_TPC', '_perp_dist_any_wall', 
#'_going_upwards', '_dist_to_closest_particle']

#Plots from ERTool cosmic ana output
def mediumLevelPlotsCompareSamples(keylist):
    subplots = range(221,221+len(keylist))
    ### Track length of missed protons from neutrons
    fig = plt.figure(figsize=(20,12))
    mybins = np.linspace(0,1,100)
    myquery = '_true_primary and not _reco_primary and _pdg_self == 2212 and _pdg_parent == 2112'
    for x in xrange(len(keylist)):
        mykey = keylist[x]
        myhisto = gen_histo(mykey = mykey, 
                            mytype = 'ert', 
                            myvar = '_trk_length', 
                            myquery = myquery, 
                            mybinning = mybins,
                            myunitfactor = 1.)

        plot_hist(mykey,
                  myhisto, 
                  mytitle = 'Track Length of Missed Primaries: p w/ parent n: %s'%key,
                  myxlabel = 'Track Length [cm]',
                  myylabel = 'Counts',
                  myyscale = 'linear',
                  mysubplot = subplots[x])

    ### Energy deposited by missed gammas with no parents
    fig = plt.figure(figsize=(20,12))
    mybins = mybins = np.linspace(0,500,50)
    myquery = '_true_primary and not _reco_primary and not _reco_orphan and _pdg_self == 22 and _pdg_parent == -1'
    for x in xrange(len(keylist)):
        mykey = keylist[x]
        myhisto = gen_histo(mykey = mykey, 
                            mytype = 'ert', 
                            myvar = '_energy', 
                            myquery = myquery, 
                            mybinning = mybins,
                            myunitfactor = 1.)

        plot_hist(mykey,
                  myhisto, 
                  mytitle = 'Shower Energy of Missed Primary: Gamma w/o parent: %s [50MeV cut]'%key,
                  myxlabel = 'DetProfile Shower Energy [MeV]',
                  myylabel = 'Counts',
                  myyscale = 'linear',
                  mysubplot = subplots[x])

    ### Distance to nearest TPC wall by gammas with no parents
    fig = plt.figure(figsize=(20,12))
    mybins = np.linspace(0,100,50)
    myquery = '_true_primary and not _reco_primary and not _reco_orphan and _pdg_self == 22 and _pdg_parent == -1', 
    for x in xrange(len(keylist)):
        mykey = keylist[x]

        myhisto = gen_histo(mykey = mykey, 
                            mytype = 'ert', 
                            myvar = '_perp_dist_any_wall', 
                            myquery = myquery, 
                            mybinning = mybins,
                            myunitfactor = 1.)

        plot_hist(mykey,
                  myhisto, 
                  mytitle = 'Dist to TPC Wall: Missed Gammas w/o Parent: %s [50MeV cut]'%key,
                  myxlabel = 'Perpendicular Distance to Nearest TPC wall [cm]',
                  myylabel = 'Counts',
                  myyscale = 'linear',
                  mysubplot = subplots[x])


In [ ]:
#Plots from ERTool CCSingleE ana output (including cosmic tagging)
def highLevelPlotsCompareSamples(keylist):
    fidvolcut = '_x_vtx > 10 and _x_vtx < 246.35 and _y_vtx > -106.5 and _y_vtx < 106.5 and _z_vtx > 10 and _z_vtx < 1026.8'
    defaultcut = '_longestTrackLen < 100.'

    subplots = range(221,221+len(keylist))
    ### CCSingleE Cosmic MIDs
    fig = plt.figure(figsize=(20,12))
    mybins = np.linspace(0,3,32)
    myquery = ''
    for x in xrange(len(keylist)):
        mykey = keylist[x]
        myhisto = gen_histo(mykey = mykey, 
                            mytype = 'ccs', 
                            myvar = '_e_nuReco', 
                            myquery = myquery, 
                            mybinning = mybins,
                            myunitfactor = 1000.,
                            myweight = cosWeight(mykey))

        plot_hist(mykey,
                  myhisto, 
                  mytitle = 'CCSingleE: Cosmic MIDs (6.6e20 Scaled): %s'%mykey,
                  myxlabel = 'Reconstructed Neutrino Energy [GeV]',
                  myylabel = 'Counts',
                  myyscale = 'linear',
                  myymax = 1200.,
                  mysubplot = subplots[x],
                  myweight = cosWeight(mykey))

In [ ]:
lowLevelPlotsCompareSamples(dfs.keys())
mediumLevelPlotsCompareSamples(dfs.keys())
highLevelPlotsCompareSamples(dfs.keys())